In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt

DEFAULT_SEED = 42

from gaussian import MultivariateNormal, VarianceExploding
from numerical import BroydenSolver
from utils import cube_vertices, solve_numerical_scheme, solve_flow

In [ ]:
norms = cube_vertices(dim=2)
mix = VarianceExploding(norms)
tf = 2.0

timesteps_list = [10, 20, 40, 80, 160, 320]

### solve_ivp (VE)

In [ ]:
prior = MultivariateNormal(mix.dim, cov=mix.added_noise_sq(tf))
x_solve_ivp, _, _ = solve_flow(mix, prior, tf=tf)

### Implicit Euler (VE) per timesteps (10, 20, 40, 80, 160, 320)

In [ ]:
ve_dict = {}
for n_ts in timesteps_list:
    print(f"{n_ts} steps")
    _, x, _ = solve_numerical_scheme(solver=BroydenSolver, mix=mix, n_samples=5000, t_min=1e-6, tf=tf, n_timesteps=n_ts, linear_ts=False)
    _, x_linear, _ = solve_numerical_scheme(solver=BroydenSolver, mix=mix, n_samples=5000, t_min=1e-6, tf=tf, n_timesteps=n_ts, linear_ts=True)
    ve_dict[n_ts] = (x[:,-1,:], x_linear[:,-1,:])

### MSE solve_ivp vs implicit Euler (VE) per timesteps (10, 20, 40, 80, 160, 320) and per timesteps modality (EDM, linear)

In [ ]:
for ts in timesteps_list:
    mse_edm = ((x_solve_ivp - ve_dict[ts][0])**2).mean()
    mse_linear = ((x_solve_ivp - ve_dict[ts][1])**2).mean()
    print(f"Steps: {ts}, MSE EDM: {mse_edm}, MSE linear: {mse_linear}")

### Plots

In [ ]:
def make_plots(x_solve_ivp, ve_dict):
    fig, axes = plt.subplots(2, len(timesteps_list)+1, figsize=(15, 4), dpi=300)

    for ax, col in zip(axes[0], ['solve_ivp', *timesteps_list]):
        ax.set_title(col)

    scheduling = ['Linear', 'EDM']
    for ax, row in zip(axes[:,0], scheduling):
        ax.set_ylabel(row, rotation=90, size='large')

    for i in range(len(scheduling)):
        axes[i, 0].scatter(*x_solve_ivp.T, s=1)
        for j, n_ts in enumerate(timesteps_list):
            axes[i, j+1].scatter(*ve_dict[n_ts][i].T, s=1)

    return fig, axes

In [ ]:
make_plots(x_solve_ivp, ve_dict);